## Estructura multicapa unidimensional

In [ ]:
import numpy as np  
import matplotlib as mpl 
import matplotlib.pyplot as plt  
import math
from cmath import sqrt
%matplotlib inline
from matplotlib import pyplot as plt
from ipywidgets import interact

In [ ]:
# Parámetros
PPW = 50  # Points Per Wavelenght


In [ ]:
a = 0
b = 22 / 100           #Se define una cavidad de 22 cm, para darle espacio a la sección de vacío y la división de tejidos aproximadamente de 12 cm

L = b-a
fmax = 3e9 
w=3e9
e0 = 8.8541878176e-12   #Permitividad del vacio F/m
m0 = (4e-7)*np.pi       #Permeabilidad del vacío (T*m)/A


mr = 1                  #Permeabilidad relativa en el vacío
er = 1                  #Permitividad relativa en el vacío

#--------------------------------------------------------------

mu = mr * m0            #Permeabilidad del tejido


In [ ]:
#Permitividades, ecuación de debye

#PIEL
erp = 4+(32/(1+1j*w*7.23e-12)) 

#TEJIDO ADIPOSO
era = 3.140+(1.708/(1+1j*w*(14.65e-12)**(1-0.061))) 

#TEJIDO GLANDULAR
erg = 7.821 +(41.48/(1+1j*w*(10.66e-12)**(1-0.047))) 

#TUMOR
ergc = 9.058+(51.31/(1+1j*w*(10.84e-12)**(1-0.022))) 

#TEJIDO MUSCULAR
erm = 4+(50/(1+1j*w*(7.23e-12)**(1-0.10))) 

#Permitividades del tejido--------------------------------------

eps = er * e0           #Vacío
epsp = erp *e0          #Piel
epsa = era *e0          #Tejido adiposo
epsg = erg *e0          #Tejido glandular
epsgc = ergc *e0        #Tumor
epsm = erm *e0          #Tejido muscular



In [ ]:
n0 = sqrt(m0/e0)
npp = sqrt(mr/erp)*n0
na = sqrt(mr/era)*n0
ng = sqrt(mr/erg)*n0
nt = sqrt(mr/ergc)*n0
nm = sqrt(mr/erm)*n0


In [ ]:
#Cavidad resonante de tamaño L
#División de tamaños de los tipos de tejido en cm

LP=1.5 / 1000                #Piel 
LA=1.5   / 100               #Tejido adiposo 
LG=5.9 / 100              #Tejido glandular
#L4=3.693 / 100             #Tumor
L4=float(input("Ingrese un tamaño entre 0.03 cm y 5 cm para el tumor: "))/100
L5=0.5 / 100               #Tejido muscular
L0=L-(LP+LA+LG+L4+L5)                #Vacío

#Posiciones donde termina cada tipo de tejido

PV=L0
PP=L0+LP              #Posición Piel
PA=PP+LA              #Posición tejido adiposo
PG=PA+LG              #Posición tejido glandular
PT=PG+L4              #Posición tumor
PM=PT+L5              #Posición tejido muscular


In [ ]:
#Velocidad de la onda electromagnética, lambda y dz asociado a cada tipo de tejido

#Vacío

c0 = 1/(np.sqrt(e0*m0))
lam_min = c0/fmax
dz_v = lam_min/PPW

#Piel
c_p=abs(c0/np.sqrt(erp*mr))
lam_min_p=c_p/fmax
dz_p=lam_min_p/PPW

#Tejido adiposo

c_a=abs(c0/np.sqrt(era*mr))
lam_min_a=c_a/fmax
dz_a=lam_min_a/PPW

#Tejido glandular

c_g=abs(c0/np.sqrt(erg*mr))
lam_min_g=c_g/fmax
dz_g=lam_min_g/PPW

#Tumor

c_t=abs(c0/np.sqrt(ergc*mr))
lam_min_t=c_t/fmax
dz_t=lam_min_t/PPW

#Tejido muscular

c_m=abs(c0/np.sqrt(erm*mr))
lam_min_m=c_m/fmax
dz_m=lam_min_m/PPW



In [ ]:
#Intervalos asociados a su dz particular en cada tipo de tejido

#Vacío------------------------------
I_v=np.ceil(L0/(dz_v))+1
I_v = I_v.astype(int)
I_v_2 = 2*(I_v-1)                         #Hago esto en cada tejido para los ciclos for de cada matriz
#-----------------------------------

#Piel-------------------------------
I_piel=np.ceil(LP/dz_p)+1
I_piel=I_piel.astype(int)
I_piel_2=2*(I_piel-1)
#-----------------------------------

#Tejido adiposo---------------------
I_adiposo=np.ceil(LA/dz_a)+1
I_adiposo=I_adiposo.astype(int)
I_adiposo_2=2*(I_adiposo-1)
#-----------------------------------

#Tejido glandular-------------------
I_glandular=np.ceil(LG/dz_g)+1
I_glandular=I_glandular.astype(int)
I_glandular_2=2*(I_glandular-1)
#-----------------------------------

#Tumor------------------------------
I_tumor=np.ceil(L4/dz_t)+1
I_tumor=I_tumor.astype(int)
I_tumor_2=2*(I_tumor-1)
#-----------------------------------

#Tejido muscular--------------------
I_muscular=np.ceil(L5/dz_m)+1
I_muscular=I_muscular.astype(int)
I_muscular_2=2*(I_muscular-1)
#-----------------------------------

#Intervalo donde termina cada tipo de tejido teniendo en cuenta los demás

#Piel----------------
I_p=I_v_2+I_piel_2
I_p=I_p.astype(int)
#--------------------

#Tejido adiposo------
I_a=I_p+I_adiposo_2
I_a=I_a.astype(int)
#--------------------

#Tejido glandular----
I_g=I_a+I_glandular_2
I_g=I_g.astype(int)
#--------------------

#Tumor---------------
I_t=I_g+I_tumor_2
I_t=I_t.astype(int)
#--------------------

#Tejido muscular-----
I_m=I_t+I_muscular_2
I_m=I_m.astype(int)
#--------------------

#Total de intervalos---------------------------------
I=I_v+I_piel+I_adiposo+I_glandular+I_tumor+I_muscular
#----------------------------------------------------


In [ ]:
#Posiciones
#Campo Eléctrico

#len-->I+1

z_1=np.linspace(a,PV,I_v);  dz1=z_1[1]-z_1[0]
zh1=0.5*(z_1[0:-1]+z_1[1::]); dzh1=zh1[1]-zh1[0]

z_2=np.linspace(PV,PP,I_piel); dz2=z_2[1]-z_2[0]
zh2=0.5*(z_2[0:-1]+z_2[1::]); dzh2=zh2[1]-zh2[0]

z_3=np.linspace(PP,PA,I_adiposo); dz3=z_3[1]-z_3[0]
zh3=0.5*(z_3[0:-1]+z_3[1::]); dzh3=zh3[1]-zh3[0]

z_4=np.linspace(PA,PG,I_glandular); dz4=z_4[1]-z_4[0]
zh4=0.5*(z_4[0:-1]+z_4[1::]); dzh4=zh4[1]-zh4[0]

z_5=np.linspace(PG,PT,I_tumor); dz5=z_5[1]-z_5[0]
zh5=0.5*(z_5[0:-1]+z_5[1::]); dzh5=zh5[1]-zh5[0]

z_6=np.linspace(PT,PM,I_muscular); dz6=z_6[1]-z_6[0] 
zh6=0.5*(z_6[0:-1]+z_6[1::]); dzh6=zh6[1]-zh6[0]


z=np.hstack((z_1,z_2[1::],z_3[1::],z_4[1::],z_5[1::],z_6[1::]))

#Campo Magnético

#len-->I

z2=0.5*(z[0:-1]+z[1::])

Itotal=I_m+1



In [ ]:
#Creación de matrices

B=np.zeros((Itotal,Itotal), dtype=np.complex_)  #Matriz de rotacionales

V1=np.zeros((Itotal,Itotal), dtype=np.complex_)  #Submatriz vacio
P1=np.zeros((Itotal,Itotal),dtype=np.complex_)   #Submatriz piel
Ad1=np.zeros((Itotal,Itotal),dtype=np.complex_)  #Submatriz tejido adiposo
G1=np.zeros((Itotal,Itotal),dtype=np.complex_)   #Submatriz tejido glandular
Tu1=np.zeros((Itotal,Itotal),dtype=np.complex_)  #Submatriz tumor
M1=np.zeros((Itotal,Itotal),dtype=np.complex_)

#Vectores asociados a los campos

E1=np.zeros((len(z),1),dtype=np.complex_)     #Campo eléctrico(E) con la solución del vector propio (v.p) de la primera columna
H1=np.zeros((len(z2),1),dtype=np.complex_)       #Campo magnético(H) con la solución del v.p de la primera columna

len(E1),len(H1)

In [ ]:
def vacio1():   
#------------------vacio--------------------    
    
    for v1 in range(0,I_v_2+1):
        
        if v1%2 == 0:
        
            if v1 == 0:
                  
                V1[v1][v1+1] = -2*m0*c0
                V1[v1][v1+2] = -1
                
            
            if v1 == I_v_2:
            
                V1[v1][v1-4] =  -(1/2) 
                V1[v1][v1-2] = (1) 
            
            #else:
            
            V1[v1][v1-1] =  (1/dz1)*n0*c0
            V1[v1][v1+1] = (-1/dz1)*n0*c0
                
        elif v1%2 != 0:
            
            V1[v1][v1-1] =  (1/dzh1)*(1/n0)*c0
            V1[v1][v1+1] = (-1/dzh1)*(1/n0)*c0
            
            
    return(V1)
 
def piel1():    
#------------------piel---------------------  

    for p1 in range(I_v_2,I_p+1):
        
        if p1%2 == 0:
        
            if p1 == I_p:
            
                P1[p1][p1-4] = -(1/2) 
                P1[p1][p1-2] = (1) 
            
            if p1 == I_v_2:
            
                P1[p1][p1+2] = (1) 
                P1[p1][p1+4] = -(1/2) 
        
            #else:
            
            P1[p1][p1-1] =  (1/dz2)*npp*c_p
            P1[p1][p1+1] = (-1/dz2)*npp*c_p
                
        elif p1%2 != 0:
            
            P1[p1][p1-1] =  (1/dzh2)*(1/npp)*c_p
            P1[p1][p1+1] = (-1/dzh2)*(1/npp)*c_p
            
    return(P1)    
#-------------tejido adiposo----------------

def adiposo1():
    
    for a1 in range(I_p,I_a+1):
        
        if a1%2 == 0:
            
            if a1 == I_a:
            
                Ad1[a1][a1-4] = -(1/2)
                Ad1[a1][a1-2] = (1)
                
            
            if a1 == I_p:
            
                Ad1[a1][a1+2] = (1) 
                Ad1[a1][a1+4] = -(1/2) 
            

            #else:
            
            Ad1[a1][a1-1] =  (1/dz3)*na*c_a
            Ad1[a1][a1+1] = (-1/dz3)*na*c_a
                
        elif a1%2 != 0:
            
            Ad1[a1][a1-1] =  (1/dzh3)*(1/na)*c_a
            Ad1[a1][a1+1] = (-1/dzh3)*(1/na)*c_a
            

    return(Ad1)
#------------tejido glandular---------------

def glandular1():
    
    for g1 in range(I_a,I_g+1):
        
        if g1%2 == 0:
            
            if g1 == I_g:
            
                G1[g1][g1-4] = -(1/2)
                G1[g1][g1-2] = (1)
            
            if g1 == I_a:
            
                G1[g1][g1+2] = (1) 
                G1[g1][g1+4] = -(1/2) 
            
            
            #else:
              
            G1[g1][g1-1] =  (1/dz4)*ng*c_g
            G1[g1][g1+1] = (-1/dz4)*ng*c_g
                
        elif g1%2 != 0:
            
            G1[g1][g1-1] =  (1/dzh4)*(1/ng)*c_g
            G1[g1][g1+1] = (-1/dzh4)*(1/ng)*c_g

    return(G1)       
#-------------------tumor-------------------

def tumor1():
    
    for t1 in range(I_g,I_t+1):
        
        if t1%2 == 0:
        
            if t1 == I_t:
            
                Tu1[t1][t1-4] = -(1/2)
                Tu1[t1][t1-2] = (1)
            
            if t1 == I_g:
            
                Tu1[t1][t1+2] = (1) 
                Tu1[t1][t1+4] = -(1/2) 
            
            
            #else:
                  
            Tu1[t1][t1-1] =  (1/dz5)*nt*c_t
            Tu1[t1][t1+1] = (-1/dz5)*nt*c_t
                
        elif t1%2 != 0:
            
            Tu1[t1][t1-1] =  (1/dzh5)*(1/nt)*c_t
            Tu1[t1][t1+1] = (-1/dzh5)*(1/nt)*c_t
            
    return(Tu1)   
#----------------tejido muscular---------------        

def muscular1():
        
    for m1 in range(I_t,I_m+1):
        
        if m1%2 == 0:
            
            if m1 == I_m:
            
                M1[m1][m1-1] = 2*m0*c0
                M1[m1][m1-2] = -1
                
            
            elif m1 == I_t:
            
                M1[m1][m1+2] = (1) 
                M1[m1][m1+4] = (-1/2) 
            

            else:
            
                M1[m1][m1-1] =  (1/dz6)*nm*c_m
                M1[m1][m1+1] = (-1/dz6)*nm*c_m
                
        elif m1%2 != 0:
            
            M1[m1][m1-1] =  (1/dzh6)*(1/nm)*c_m
            M1[m1][m1+1] = (-1/dzh6)*(1/nm)*c_m
                
    return(M1)

In [ ]:
B = vacio1()+piel1()+adiposo1()+glandular1()+tumor1()+muscular1()

In [ ]:
#Valores y vectores propios

Lp,Vp=np.linalg.eig(B)  #Lp--> valores propios, V--> vectores propios


### Índices de los valores propios

Se organiza la parte imaginaria de los valores propios de menor a mayor y se obtienen sus índices para poder asociarlos a los vectores propios.

In [ ]:
idx = np.argsort(Lp.imag)   #Toma los valores propios y los organiza de menor a mayor y se le asigna un índice a cada uno
L1 = Lp[idx]                #Se organizan los valores propios usando el índice que se tomó en el paso anterior
Vp1 = Vp[:,idx]            #Toma el índice asociado a los valores propios y los asigna a cada columna de la matriz de vectores propios para poder organizarlos


In [ ]:
L11=L1.imag


In [ ]:
for cero in zip(*np.where(L11<1)):    #Guardo el indice del valor propio=0 (el ultimo)
    cero
cero=cero[0]

if L11[cero+1]<1:
    cero=cero+1


In [ ]:
L2= L11[cero::]      #Tomar los valores positivos de los valores propios ordenados
Vp2= Vp1[:,cero::]
lenL2 = len(L2)


In [ ]:
fnum = np.zeros(lenL2)
for j in range(0,lenL2):
    
    fnum[j] = (L2[j])/(2*np.pi)
    
print("Rango de frecuencias comprendido entre: " + np.format_float_scientific(fnum[1],precision = 3, exp_digits=2) + " Hz y " + np.format_float_scientific(fnum[-1],precision = 3, exp_digits=2) + " Hz")
frecu=int(input("Ingrese un numero entre 0 y " + str(len(L2)-1)  + " para indicar el índice de la frecuencia a trabajar dentro del rango presentado: "))

frecuencia=L2[frecu]/(2*np.pi)

print("Se observarán los campos electromagnéticos asociados a la frecuencia: " + np.format_float_scientific(frecuencia,precision = 3, exp_digits=2) + " Hz.")



In [ ]:
for num in range(0,len(str(L2[frecu]))):
    if str(L2[frecu])[num]==".":
        long=num


In [ ]:
lista=[]
for i in range(5):
    lista.append(frecu+(i-2)) #Guardo el indice de los errores que estan por debajo del % de error aceptable

nlen=len(lista)


In [ ]:
val_prop_real=[]
vec_prop_real=[]
vec=Vp2

In [ ]:
for l in range(0,nlen):
    val_prop_real.append(L2[lista[l]])
    
vec_prop_real=Vp2[:,lista[0]:lista[nlen-1]+1]

In [ ]:
L3= (val_prop_real)             #Como los valores propios ordenados ya se habían ordenado y se habían tomado solo los positivos, tomamos los primeros 34 valores de k (positivos)
Vp3=(vec_prop_real)      #Los vectores propios ordenados incluyen negativos y positivos entonces tomamos las columnas correspondientes a los valores propios tomados antes


In [ ]:
Vp3_E=Vp3[::2]
Vp3_H=Vp3[1::2]
fila=len(Vp3_E[1,:])
E=np.zeros((len(z),len(Vp3[1,:])),dtype=np.complex_)
H=np.zeros((len(z2),len(Vp3[1,:])),dtype=np.complex_)


In [ ]:
for i in range(0,fila):
    E[:,i]=Vp3_E[:,i]
    H[:,i]=Vp3_H[:,i]

In [ ]:
### Crear figura y ejes

def test(i):
    fig = plt.figure(figsize=(14, 20))
    aE1 = fig.add_subplot(321)
    aH1 = fig.add_subplot(322)
    aE2 = fig.add_subplot(323)
    aH2 = fig.add_subplot(324)
    aE3 = fig.add_subplot(325)
    aH3 = fig.add_subplot(326)

# Plot

    
    aE1.plot(z, E[:,i].real,color='blue', linewidth=1)
    aE1.title.set_text("Campo Eléctrico")
    aH1.plot(z2, H[:,i].imag,color='red', linewidth=1)
    aH1.title.set_text("H")


    aE2.plot(z, E[:,i].imag,color='blue', linewidth=1)
    aH2.plot(z2, H[:,i].real,color='red', linewidth=1)


    aE3.plot(z, abs(E[:,i]),color='blue', linewidth=1)
    aH3.plot(z2, abs(H[:,i]),color='red', linewidth=1)
    
    plt.autoscale(enable=True, axis='y')
    plt.autoscale(enable=True, axis='x')

    fig.suptitle('Campos E y H',fontweight ="bold")
    plt.xlabel('z')
    plt.ylabel('Campo E real')
    

    return()


In [ ]:
#Indices de refracción --> n=c/v (c--> velocidad de la onda en el vacío, v--> velocidad de la onda en el material)

na=1                 #Vacio
n1=c0/c_p            #Piel
n2=c0/c_a            #T. adiposo
n3=c0/c_g            #T. glandular
n4=c0/c_t            #Tumor
n5=c0/c_m            #T. muscular
nb=1

#Coeficientes de reflexión

pp1=(na-n1)/(na+n1)  #vacio
pp2=(n1-n2)/(n1+n2)  #piel
pp3=(n2-n3)/(n2+n3)  #Tejido adiposo
pp4=(n3-n4)/(n3+n4)  #Tejido glandular
pp5=(n4-n5)/(n4+n5)  #Tumor
pp6=(n5-nb)/(n5+nb)  #Tejido muscular

#Números de onda
#k=w/c
#k=2pi/lam
#lam=c/f

k1=(L3)/c_p
k2=(L3)/c_a
k3=(L3)/c_g
k4=(L3)/c_t
k5=(L3)/c_m

#Respuesta de reflexión

Gamma6=pp6
Gamma5=(pp5+Gamma6*np.exp(-2*1j*k5*L5))/(1+pp5*Gamma6*np.exp(-2*1j*k5*L5))
Gamma4=(pp4+Gamma5*np.exp(-2*1j*k4*L4))/(1+pp4*Gamma5*np.exp(-2*1j*k4*L4))
Gamma3=(pp3+Gamma4*np.exp(-2*1j*k3*LG))/(1+pp3*Gamma4*np.exp(-2*1j*k3*LG))
Gamma2=(pp2+Gamma3*np.exp(-2*1j*k2*LA))/(1+pp2*Gamma3*np.exp(-2*1j*k2*LA))
Gamma1=(pp1+Gamma2*np.exp(-2*1j*k1*LP))/(1+pp1*Gamma2*np.exp(-2*1j*k1*LP))

#Reflectancia

R_Gamma1=(abs(Gamma1))**2
R_Gamma2=(abs(Gamma2))**2
R_Gamma3=(abs(Gamma3))**2
R_Gamma4=(abs(Gamma4))**2
R_Gamma5=(abs(Gamma5))**2
R_Gamma6=(abs(Gamma6))**2


In [ ]:
frecuenciaL2=L2/(2*np.pi)


Perp = 4+(32/(1+1j*L2*7.23e-12)) 
Pera = 3.140+(1.708/(1+1j*L2*(14.65e-12)**(1-0.061))) 
Perg = 7.821 +(41.48/(1+1j*L2*(10.66e-12)**(1-0.047))) 
Pergc = 9.058+(51.31/(1+1j*L2*(10.84e-12)**(1-0.022))) 
Perm = 4+(50/(1+1j*L2*(7.23e-12)**(1-0.10)))

## Permitividades en función de la frecuencia

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(8,8))

plt.plot(abs(L2),abs(Perp), label="Piel")
plt.plot(abs(L2),abs(Pera), label="Tejido adiposo")
plt.plot(abs(L2),abs(Perg), label="Tejido glandular")
plt.plot(abs(L2),abs(Pergc), label="Tumor")
plt.plot(abs(L2),abs(Perm), label="Tejido muscular")
plt.legend()
axes.set_title("Permitividad en función de la frecuencia",fontweight ="bold")
plt.xlabel("Frecuencias (Hz)")
plt.ylabel("|Permitividades complejas|")
plt.plot()

## Gráficas campos EM

In [ ]:
interact(test,i=(0,fila-1))
print("Campos electromagnéticos en un rango de frecuencias de: " + np.format_float_scientific((L3[0])/(2*np.pi),precision = 3, exp_digits=2) + " Hz y " + np.format_float_scientific(L3[len(L3)-1]/(2*np.pi),precision = 3, exp_digits=2) + " Hz." )
print("Los tamaños son --> Vacío= " +str(round(L0, 3))+ "m, Piel= " +str(round(LP,3))+ "m, Tejido adiposo= " +str(round(LA,3))+ "m, Tejido glandular= " +str(round(LG,3))+ "m,\n Tumor= " +str(round(L4,4))+ "m, Tejido muscular= " +str(round(L5,3))+ "m.")
print("Las posiciones hasta donde llega cada sección son: " +str(round(PV, 3))+ "m, Piel= " +str(round(PP,3))+ "m, Tejido adiposo= " +str(round(PA,3))+ "m,\n Tejido glandular= " +str(round(PG,3))+ "m, Tumor= " +str(round(PT,3))+ "m, Tejido muscular= " +str(round(PM,3))+ "m.")